In [1]:
# Import necessary packages
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
# bash Linux commands for checking the allotted GPU
!nvidia-smi
! env | grep CUDA

Mon Oct 24 07:44:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Hyperparameters
input_size=28*28
num_classes=10
num_epochs=10
batch_size=128
learning_rate=0.001
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device available: {device}')

Device available: cuda


In [4]:
# Dataset
train_dataset=torchvision.datasets.MNIST(root='./data',train=True,download=True,transform=transforms.ToTensor())

test_dataset=torchvision.datasets.MNIST(root='./data',train=False,download=True,transform=transforms.ToTensor())

In [5]:
# Dataloaders
train_dataloader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_dataloader=DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [6]:
len(train_dataloader)

469

In [7]:
# Logistic Regression model
model= nn.Linear(input_size,num_classes).to(device)

In [8]:
# Loss and optimizer
# Note : nn.CrossEntropyLoss() computes softmax internally
criterion=nn.CrossEntropyLoss()

optimizer=optim.SGD(model.parameters(),lr=learning_rate)

In [9]:
# Train the model
total_step=len(train_dataloader)
start_time=time.time()
for epoch in range(num_epochs):
  for i,(images,labels) in enumerate(train_dataloader):
     # Reshape the images to 1D vector
     images=images.reshape(-1,input_size).to(device)
     labels=labels.to(device)
     # Forward pass
     prediction=model(images)
     # compute the loss
     loss=criterion(prediction,labels)
     # Backward pass
     # zero the gradients
     optimizer.zero_grad()
     # perform backpropagation
     loss.backward()
     # update the weights
     optimizer.step()
     if(i+1)%100 == 0:
       print(f'Epochs :{epoch+1}/{num_epochs}  Steps: {i+1}/{total_step} Loss: {loss.item():.4f}')
end_time=time.time()
print(f'Training took {end_time-start_time:.2f} s')
      



Epochs :1/10  Steps: 100/469 Loss: 2.2103
Epochs :1/10  Steps: 200/469 Loss: 2.1193
Epochs :1/10  Steps: 300/469 Loss: 2.0285
Epochs :1/10  Steps: 400/469 Loss: 1.9193
Epochs :2/10  Steps: 100/469 Loss: 1.8217
Epochs :2/10  Steps: 200/469 Loss: 1.7086
Epochs :2/10  Steps: 300/469 Loss: 1.7239
Epochs :2/10  Steps: 400/469 Loss: 1.6306
Epochs :3/10  Steps: 100/469 Loss: 1.6209
Epochs :3/10  Steps: 200/469 Loss: 1.4404
Epochs :3/10  Steps: 300/469 Loss: 1.5304
Epochs :3/10  Steps: 400/469 Loss: 1.3874
Epochs :4/10  Steps: 100/469 Loss: 1.3133
Epochs :4/10  Steps: 200/469 Loss: 1.2863
Epochs :4/10  Steps: 300/469 Loss: 1.2750
Epochs :4/10  Steps: 400/469 Loss: 1.2153
Epochs :5/10  Steps: 100/469 Loss: 1.1916
Epochs :5/10  Steps: 200/469 Loss: 1.1962
Epochs :5/10  Steps: 300/469 Loss: 1.2139
Epochs :5/10  Steps: 400/469 Loss: 1.1256
Epochs :6/10  Steps: 100/469 Loss: 1.0946
Epochs :6/10  Steps: 200/469 Loss: 1.0942
Epochs :6/10  Steps: 300/469 Loss: 1.0142
Epochs :6/10  Steps: 400/469 Loss:

In [10]:
# Inferencing phase
with torch.no_grad():
  correct=0
  total=0
  for images,labels in test_dataloader:
    images=images.reshape(-1,input_size).to(device)
    labels=labels.to(device)
    outputs=model(images)
    _,predictions=torch.max(outputs.data,axis=1)
    total+=labels.size(0)
    correct+=(predictions==labels).sum()
  print(f'Test Accuracy of the logistic regression model:{(correct/total)*100:.2f}')

Test Accuracy of the logistic regression model:84.53


In [11]:
# save the model
torch.save(model.state_dict(),'model.ckpt')